In [3]:
import numpy as np
from numpy import dstack
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
import os

In [14]:
def load_file(filepath):
    import pandas as pd
    return pd.read_csv(filepath, delim_whitespace=True, header=None, on_bad_lines='skip').to_numpy()


def load_group(filenames, prefix=''):
    loaded = []
    for name in filenames:
        data = load_file(prefix + name)
        loaded.append(data)
    return dstack(loaded)

def load_dataset_group(group, prefix=''):
    filepath = prefix + group + '/Inertial Signals/'

    filenames = [
        'total_acc_x_', 'total_acc_y_', 'total_acc_z_',
        'body_acc_x_', 'body_acc_y_', 'body_acc_z_',
        'body_gyro_x_', 'body_gyro_y_', 'body_gyro_z_'
    ]
    filenames = [f + group + '.txt' for f in filenames]

    X = load_group(filenames, filepath)
    y = load_file(prefix + group + '/y_' + group + '.txt')
    return X, y


In [15]:
X_train, Y_train = load_dataset_group('train', 'UCIDataset/')
X_test, Y_test = load_dataset_group('test', 'UCIDataset/')

Y_train = Y_train - 1
Y_test = Y_test - 1
Y_train = to_categorical(Y_train)
Y_test = to_categorical(Y_test)

print('X_train.shape : ', X_train.shape)  # Expected: (7352, 128, 9)
print('Y_train.shape : ', Y_train.shape)  # Expected: (7352, 6)

# Add channel dimension
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], X_train.shape[2], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], X_test.shape[2], 1))


C:\Users\DG-MEC\AppData\Local\Temp\ipykernel_1920\759800176.py:3: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  return pd.read_csv(filepath, delim_whitespace=True, header=None, on_bad_lines='skip').to_numpy()
C:\Users\DG-MEC\AppData\Local\Temp\ipykernel_1920\759800176.py:3: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  return pd.read_csv(filepath, delim_whitespace=True, header=None, on_bad_lines='skip').to_numpy()
C:\Users\DG-MEC\AppData\Local\Temp\ipykernel_1920\759800176.py:3: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  return pd.read_csv(filepath, delim_whitespace=True, header=None, on_bad_lines='skip').to_numpy()
C:\Users\DG-MEC\AppData\Local\Temp\ipykernel_1920\759800176.py:3: FutureWarning

X_train.shape :  (7352, 128, 9)
Y_train.shape :  (7352, 6)


C:\Users\DG-MEC\AppData\Local\Temp\ipykernel_1920\759800176.py:3: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  return pd.read_csv(filepath, delim_whitespace=True, header=None, on_bad_lines='skip').to_numpy()


In [18]:
model = Sequential()

# First block
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(128, 9, 1)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

# Second block
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

# Fully connected layers
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(6, activation='softmax'))


model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 128, 9, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 128, 9, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 64, 4, 64)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 64, 4, 64)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 64, 4, 64)      │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 32, 2, 64)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 32, 2, 64)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 100)            │       409,700 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 6)              │           606 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 466,050 (1.78 MB)

 Trainable params: 466,050 (1.78 MB)

 Non-trainable params: 0 (0.00 B)

In [19]:
checkpoint = ModelCheckpoint("har_weights.h5", monitor='val_accuracy', verbose=1,
                             save_best_only=True, save_weights_only=False, mode='auto')


In [22]:
history = model.fit(
    X_train, Y_train,
    validation_data=(X_test, Y_test),
    epochs=10,
    batch_size=20,
    callbacks=[checkpoint],
    verbose=1
)


Epoch 1/10
367/368 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.9534 - loss: 0.1068
Epoch 1: val_accuracy improved from 0.85002 to 0.88904, saving model to har_weights.h5


368/368 ━━━━━━━━━━━━━━━━━━━━ 21s 58ms/step - accuracy: 0.9533 - loss: 0.1068 - val_accuracy: 0.8890 - val_loss: 0.5127
Epoch 2/10
368/368 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.9443 - loss: 0.1187
Epoch 2: val_accuracy improved from 0.88904 to 0.90567, saving model to har_weights.h5


368/368 ━━━━━━━━━━━━━━━━━━━━ 21s 58ms/step - accuracy: 0.9443 - loss: 0.1187 - val_accuracy: 0.9057 - val_loss: 0.5151
Epoch 3/10
367/368 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 0.9621 - loss: 0.0808
Epoch 3: val_accuracy improved from 0.90567 to 0.91042, saving model to har_weights.h5


368/368 ━━━━━━━━━━━━━━━━━━━━ 21s 58ms/step - accuracy: 0.9621 - loss: 0.0808 - val_accuracy: 0.9104 - val_loss: 0.6823
Epoch 4/10
367/368 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.9727 - loss: 0.0636
Epoch 4: val_accuracy did not improve from 0.91042
368/368 ━━━━━━━━━━━━━━━━━━━━ 22s 60ms/step - accuracy: 0.9727 - loss: 0.0637 - val_accuracy: 0.9019 - val_loss: 0.6602
Epoch 5/10
367/368 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.9719 - loss: 0.0672
Epoch 5: val_accuracy did not improve from 0.91042
368/368 ━━━━━━━━━━━━━━━━━━━━ 23s 62ms/step - accuracy: 0.9719 - loss: 0.0672 - val_accuracy: 0.8911 - val_loss: 0.6338
Epoch 6/10
367/368 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.9690 - loss: 0.0792
Epoch 6: val_accuracy did not improve from 0.91042
368/368 ━━━━━━━━━━━━━━━━━━━━ 23s 61ms/step - accuracy: 0.9690 - loss: 0.0792 - val_accuracy: 0.8884 - val_loss: 0.5924
Epoch 7/10
367/368 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - accuracy: 0.9728 - loss: 0.0592
Epoch 7: val_accuracy i

368/368 ━━━━━━━━━━━━━━━━━━━━ 23s 63ms/step - accuracy: 0.9729 - loss: 0.0592 - val_accuracy: 0.9243 - val_loss: 0.6487
Epoch 8/10
367/368 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.9782 - loss: 0.0472
Epoch 8: val_accuracy did not improve from 0.92433
368/368 ━━━━━━━━━━━━━━━━━━━━ 38s 55ms/step - accuracy: 0.9782 - loss: 0.0472 - val_accuracy: 0.9111 - val_loss: 0.7022
Epoch 9/10
367/368 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.9755 - loss: 0.0583
Epoch 9: val_accuracy did not improve from 0.92433
368/368 ━━━━━━━━━━━━━━━━━━━━ 21s 57ms/step - accuracy: 0.9755 - loss: 0.0583 - val_accuracy: 0.9162 - val_loss: 0.4071
Epoch 10/10
367/368 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 0.9776 - loss: 0.0554
Epoch 10: val_accuracy improved from 0.92433 to 0.92671, saving model to har_weights.h5


368/368 ━━━━━━━━━━━━━━━━━━━━ 22s 59ms/step - accuracy: 0.9776 - loss: 0.0554 - val_accuracy: 0.9267 - val_loss: 0.4808


In [23]:
loss, accuracy = model.evaluate(X_test, Y_test, verbose=1)
print(f"Test accuracy: {accuracy * 100:.2f}%")


93/93 ━━━━━━━━━━━━━━━━━━━━ 3s 28ms/step - accuracy: 0.9111 - loss: 0.5685
Test accuracy: 92.67%


In [27]:
scores = model.evaluate(X_test, Y_test, batch_size = 20, verbose = 1)
print("HAR - 2D CNN Test Accuracy: %.2f%%" % (scores[1]*100))

148/148 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.9113 - loss: 0.5666
HAR - 2D CNN Test Accuracy: 92.67%


In [28]:
from keras.models import load_model
model = load_model("har_weights.h5")
model.summary()


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 128, 9, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 128, 9, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 64, 4, 64)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 64, 4, 64)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 64, 4, 64)      │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 32, 2, 64)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 32, 2, 64)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 100)            │       409,700 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 6)              │           606 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 466,052 (1.78 MB)

 Trainable params: 466,050 (1.78 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)